In [3]:
import cirq
import sympy
from scipy.optimize import *
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from scipy.linalg import fractional_matrix_power
import itertools

In [4]:
cost_matrix_4 = np.array([[np.nan    , 0.35271991, 0.96262685, 0.11727604],
                          [0.13505078, np.nan    , 0.63915344, 0.43149425],
                          [0.58432224, 0.83676812, np.nan    , 0.4879146 ],
                          [0.14998587, 0.45394107, 0.2140258 , np.nan    ]])

## Аналитическое решение

In [5]:
def cost_of_permutation(cost_matrix, town_sequence):
  cost = 0.0

  for i in range(len(town_sequence)-1):
    cost += cost_matrix [town_sequence[i]] [town_sequence[i+1]]
  
  return cost

In [6]:
# Для перевода из десятичной системы счисления в факториальную 
# понадобятся факториалы, поэтому вычислим их заранее:
#
# factorials[n] == n!

factorials = [1]
for i in range(30):
  # Предполагаю, что n_towns < 30
  factorials.append(factorials[i] * (i+1))

In [7]:
def factoradic_from_perm_number(perm_number : int, factoradic_array):
  """Преобразует число в факториальную систему счисления.
  Записывает полученные цифры в factoradic_array.
  Нулевая цифра в массиве - самая значимая. Последняя цифра всегда равна 0.
  Алгоритм из https://en.wikipedia.org/wiki/Factorial_number_system#Definition """

  N = perm_number
  n_towns = len(factoradic_array)
  assert N < factorials[n_towns]

  for i in range(len(factoradic_array)):
    factoradic_array[i] = N // factorials[n_towns - i - 1]
    N %= factorials[n_towns - i - 1]
  
  return factoradic_array

In [8]:
def permutation_from_factoradic(nums : np.ndarray):
  """Перезаписывает массив nums.
  Если изначально там находилось число в факториальной системе, то потом там будет находиться перестановка.
  Города в полученной перестановке нумеруются с 0. 
  Алгоритм из https://en.wikipedia.org/wiki/Lehmer_code#Encoding_and_decoding """
  for i in range(len(nums)-2, -1, -1):
    for j in range(i+1, len(nums)):
      if(nums[j] >= nums[i]):
        nums[j] += 1
  return nums

In [9]:
def analytical(cost_matrix):

  n_towns = len(cost_matrix)

  factoradic_array = np.ndarray(n_towns, int)

  answers_array = []

  min_cost = np.inf
  opt_perm_number = np.nan

  for perm_number in range(factorials[n_towns]):
    town_sequence = permutation_from_factoradic(
                        factoradic_from_perm_number(perm_number, factoradic_array)
                    )
    cost = cost_of_permutation(
        cost_matrix,
        town_sequence
        )

    answers_array.append([town_sequence.copy(), cost])

    if cost < min_cost:
      min_cost = cost
      opt_perm_number = perm_number

  # return {"min_cost": min_cost, 
  #         "order_of_towns": permutation_from_factoradic(
  #             factoradic_from_perm_number(opt_perm_number, factoradic_array)
  #           ) }
  return {"opt_perm_number": opt_perm_number,
          "answers_table": pd.DataFrame(answers_array, columns = ["order of towns", "cost"])}

In [10]:
anal = analytical(cost_matrix_4)
anal['answers_table'].loc[[anal['opt_perm_number']]]
# = Оптимальный путь:

,order of towns,cost
7,"[1, 0, 3, 2]",0.466353


In [11]:
anal['answers_table']

,order of towns,cost
0,"[0, 1, 2, 3]",1.479788
1,"[0, 1, 3, 2]",0.998240
2,"[0, 2, 1, 3]",2.230889
3,"[0, 2, 3, 1]",1.904483
4,"[0, 3, 1, 2]",1.210371
5,"[0, 3, 2, 1]",1.168070
6,"[1, 0, 2, 3]",1.585592
7,"[1, 0, 3, 2]",0.466353
8,"[1, 2, 0, 3]",1.340752
9,"[1, 2, 3, 0]",1.277054


In [12]:
px.scatter(anal["answers_table"]["cost"], template="plotly_dark", labels = {"index": "Номер перестановки", "value": "Cost"})

## Гейты для цепи

In [13]:
class iSwapQudits(cirq.EigenGate):
    """iSWAP for qudits, possibly raised into some power"""
    
    def __init__(self, dim, exponent = 1, global_shift = 0):
        super(iSwapQudits, self)
        self.dim = dim
        self._exponent = exponent
        self._global_shift = global_shift
        self._canonical_exponent_cached = None

    def _qid_shape_(self):
        return (self.dim, self.dim)

    def _with_exponent(self, exponent) -> 'cirq.EigenGate':
        """Return the same kind of gate, but with a different exponent.
        Child classes should override this method if they have an __init__
        method with a differing signature.
        """
        # pylint: disable=unexpected-keyword-arg
        if self._global_shift == 0:
            return type(self)(self.dim, exponent=exponent)
        return type(self)(self.dim, exponent=exponent, global_shift=self._global_shift)
        # pylint: enable=unexpected-keyword-arg

    def _eigen_shifts(self):
        """Describes the eigenvalues of the gate's matrix.
        By default, this just extracts the shifts by calling
        self._eigen_components(). However, because that method generates
        matrices it may be extremely expensive.
        Returns:
            A list of floats. Each float in the list corresponds to one of the
            eigenvalues of the gate's matrix, before accounting for any global
            shift. Each float is the θ in λ = exp(i π θ) (where λ is the
            eigenvalue).
        """
        return [0.5, -0.5, 0]

    def _eigen_components(self):
        """Describes the eigendecomposition of the gate's matrix.
        Returns:
            A list of EigenComponent tuples. Each tuple in the list
            corresponds to one of the eigenspaces of the gate's matrix. Each
            tuple has two elements. The first element of a tuple is the θ in
            λ = exp(i π θ) (where λ is the eigenvalue of the eigenspace). The
            second element is a projection matrix onto the eigenspace.
        Examples:
            The Pauli Z gate's eigencomponents are:
                [
                    (0, np.array([[1, 0],
                                  [0, 0]])),
                    (1, np.array([[0, 0],
                                  [0, 1]])),
                ]
            Valid eigencomponents for Rz(π) = -iZ are:
                [
                    (-0.5, np.array([[1, 0],
                                     [0, 0]])),
                    (+0.5, np.array([[0, 0],
                                     [0, 1]])),
                ]
            But in principle you could also use this:
                [
                    (+1.5, np.array([[1, 0],
                                     [0, 0]])),
                    (-0.5, np.array([[0, 0],
                                     [0, 1]])),
                ]
                The choice between -0.5 and +1.5 does not affect the gate's
                matrix, but it does affect the matrix of powers of the gates
                (because (x+2)*s != x*s (mod 2) when s is a real number).
            The Pauli X gate's eigencomponents are:
                [
                    (0, np.array([[0.5, 0.5],
                                  [0.5, 0.5]])),
                    (1, np.array([[+0.5, -0.5],
                                  [-0.5, +0.5]])),
                ]
        """
        return [
            ( 0.5, np.array([
                                [
                                    int(i1 == i2 and o1 == o2)
                                    - 2*int(i1 == i2 == o1 == o2)
                                    + int(i1 == o2 and i2 == o1)
                                    for i1 in range(self.dim) for o1 in range(self.dim) 
                                ] # ~ ArrayFlatten in Wolfram?
                                for i2 in range(self.dim) for o2 in range(self.dim)
                            ]) / 2
            ),
            (-0.5, np.array([
                                [
                                    int(i1 == i2 and o1 == o2)
                                    - int(i1 == o2 and i2 == o1)
                                    for i1 in range(self.dim) for o1 in range(self.dim)
                                ]
                                for i2 in range(self.dim) for o2 in range(self.dim)
                            ]) / 2
            ),
            (0,    np.array([
                                [
                                    int(i1 == i2 == o1 == o2) 
                                    for i1 in range(self.dim) for o1 in range(self.dim)
                                ]
                                for i2 in range(self.dim) for o2 in range(self.dim)
                            ])
            ),
        ]

    def _circuit_diagram_info_(self, args):
        return cirq.CircuitDiagramInfo(["↓", "↑"], 
                                    exponent = self.exponent, exponent_qubit_index = 0)

In [14]:
iSwapQudits(2)._unitary_()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
       [0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [15]:
# Для инициализации
class ModuloPlus(cirq.Gate):
    """Переводит кудит из состояния |i> в |(i+j) mod dim>,
    где dim - размерность кудита"""
    def __init__(self, dim, val):
        super(ModuloPlus, self)
        self.dim = dim
        self.val = val

    def _qid_shape_(self):
        return (self.dim,)
    
    def _unitary_(self):
        return np.array([
            [int(i == (k - self.val) % self.dim) for i in range(self.dim)] 
            for k in range(self.dim)
        ])

    def _circuit_diagram_info_(self, args):
        return f"(+{self.val})"

In [16]:
ModuloPlus(3,3)._unitary_()

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [17]:
test_circuit_1 = cirq.Circuit(
        ModuloPlus(3, 1)(cirq.LineQid(1, 3)),
        ModuloPlus(3, 2)(cirq.LineQid(2, 3)),
        iSwapQudits(3)(cirq.LineQid(0, 3), cirq.LineQid(2, 3)),
        ModuloPlus(3, -1)(cirq.LineQid(0, 3)),
        cirq.measure_each(cirq.LineQid(0, 3), cirq.LineQid(1, 3), cirq.LineQid(2, 3)))
test_circuit_1

┌──┐
0 (d=3): ───────────↓─────(+-1)───M───
                    │
1 (d=3): ───(+1)────┼M────────────────
                    │
2 (d=3): ───(+2)────↑─────M───────────
                   └──┘

In [18]:
cirq.Simulator().run(
    test_circuit_1,
    repetitions=10
)

0 (d=3)=1111111111
1 (d=3)=1111111111
2 (d=3)=0000000000

In [19]:
iSwapQudits(2, 1/2)._unitary_()

array([[1.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.70710678+0.j        ,
        0.        +0.70710678j, 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.70710678j,
        0.70710678+0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 1.        +0.j        ]])

In [20]:
cirq.ISwapPowGate(exponent = 1/2)._unitary_()

array([[1.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.70710678+0.j        ,
        0.        +0.70710678j, 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.70710678j,
        0.70710678+0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 1.        +0.j        ]])

In [21]:
class iSwapAllQudits(cirq.Gate):
    """Делает последовательно SWAP^exponent для всех n*(n-1)/2 пар кудитов"""
    def __init__(self, dim, n_qudits, exponent):
        super(iSwapAllQudits, self)
        self.n_qudits = n_qudits
        self.dim = dim
        self.exponent = exponent

    def _qid_shape_(self):
        return (self.dim,) * self.n_qudits

    def _decompose_(self, qudits):
        for n1 in range(self.n_qudits):
            for n2 in range(n1 + 1, self.n_qudits):
                yield iSwapQudits(self.dim, self.exponent)(qudits[n1], qudits[n2])

    def _circuit_diagram_info_(self, args):
        return [f"↑↓({self.exponent})"] * self.n_qudits

    def _is_parametrized_(self):
        return isinstance(self.exponent, sympy.Symbol)

    def _resolve_parameters_(self, paramresolver, recursive):
        if self._is_parametrized_():
            return iSwapAllQudits(self.dim, self.n_qudits, paramresolver.param_dict[self.exponent.name])
        else:
            return self

    def __pow__(self, power):
        return iSwapAllQudits(self.dim, self.n_qudits, power * self.exponent)

In [22]:
class CNOTSeqeunceForSingleEdge(cirq.Gate):
    """CNOTSequenceForSingeEdge(n_towns, n1, n2)(*qudits, flag_qubit)
    ставит flag_qubit в |1>, если в цепочке кудитов на каком-то месте 
    подряд стоят кудиты в состояниях |n1> и |n2>
    """
    def __init__(self, n_towns, town1, town2):
        super(CNOTSeqeunceForSingleEdge, self)
        self.n_towns = n_towns
        self.n1 = town1
        self.n2 = town2

    def _qid_shape_(self):
        return (self.n_towns,) * self.n_towns + (2,)

    def _decompose_(self, qudits):
        flag_qubit = qudits[-1]

        for town_qudit in qudits[:-2]:
            yield cirq.X.controlled(
                control_qid_shape = (self.n_towns, self.n_towns), 
                control_values = (self.n1, self.n2))(
                    town_qudit, town_qudit + 1, flag_qubit
            )

    def _circuit_diagram_info_(self, args):
        return [f"{self.n1}→{self.n2}"] * self.n_towns + ["X"]


In [23]:
test_circuit_2 = cirq.Circuit(

    ModuloPlus(3, 1)(cirq.LineQid(0, 3)),
    ModuloPlus(3, 2)(cirq.LineQid(2, 3)),
    
    CNOTSeqeunceForSingleEdge(3,0,2)(*cirq.LineQid.range(3, dimension = 3), cirq.GridQubit(0,2)),
    
    cirq.measure_each(
        *cirq.LineQid.range(3, dimension = 3), 
        cirq.GridQubit(0, 2))
)
test_circuit_2

(0, 2): ───────────X─────M───
                   │
0 (d=3): ───(+1)───0→2───M───
                   │
1 (d=3): ──────────0→2───M───
                   │
2 (d=3): ───(+2)───0→2───M───

In [24]:
cirq.Simulator().run(
    test_circuit_2,
    repetitions=10
)


(0, 2)=1111111111
0 (d=3)=1111111111
1 (d=3)=0000000000
2 (d=3)=2222222222

In [25]:
def CNOTSequenceForAllPossibleEdges(n_towns):
    """Генератор, который генерерирует все n_towns*(n_towns-1) возможных 
    CNOTSequenceForSingeEdge(n_towns, n1, n2) гейтов.
    Он перебирает все возможные пары (n1, n2) и 
    ставит cirq.GridQubit(n1, n2) в |1>, если в цепочке кудитов на каком-то месте 
    подряд стоят кудиты в состояниях |n1> и |n2>
    """
    for n1 in range(n_towns):
        for n2 in range(n_towns):
            if n1 != n2:
                yield CNOTSeqeunceForSingleEdge(n_towns, n1, n2)(
                    *cirq.LineQid.range(n_towns, dimension = n_towns),
                    cirq.GridQubit(n1, n2)
                )


In [26]:
def UndoCNOTSequences(n_towns):
    return reversed(list(itertools.chain(
        *[SingleEdgeGate._decompose_() for SingleEdgeGate in CNOTSequenceForAllPossibleEdges(n_towns)]
    )))

# Чтобы отпутать кубиты-флаги, по идее нужно проделать все CNOT в обратном порядке.
# Но, как показывает следующая ячейка, можно в том же порядке

In [27]:
(cirq.Circuit(UndoCNOTSequences(3)).unitary() ==
cirq.Circuit(CNOTSequenceForAllPossibleEdges(3)).unitary()).all()

True

In [28]:
(cirq.Circuit(UndoCNOTSequences(3)).unitary() ==
cirq.Circuit(
    reversed(list(CNOTSequenceForAllPossibleEdges(3)))
).unitary()).all()

True

In [29]:
test_circuit_3 = cirq.Circuit(

    ModuloPlus(3, 1)(cirq.LineQid(0, 3)),
    ModuloPlus(3, 2)(cirq.LineQid(2, 3)),
    
    CNOTSequenceForAllPossibleEdges(3),
    
    cirq.measure_each(
        *cirq.LineQid.range(3, dimension = 3),
        *[cirq.GridQubit(n1, n2) for n1 in range(3) for n2 in range(3) if n1 != n2]
        )
)

test_circuit_3

(0, 1): ───────────X─────M─────────────────────────────────
                   │
(0, 2): ───────────┼─────X─────M───────────────────────────
                   │     │
(1, 0): ───────────┼─────┼─────X─────M─────────────────────
                   │     │     │
(1, 2): ───────────┼─────┼─────┼─────X─────M───────────────
                   │     │     │     │
(2, 0): ───────────┼─────┼─────┼─────┼─────X─────M─────────
                   │     │     │     │     │
(2, 1): ───────────┼─────┼─────┼─────┼─────┼─────X─────M───
                   │     │     │     │     │     │
0 (d=3): ───(+1)───0→1───0→2───1→0───1→2───2→0───2→1───M───
                   │     │     │     │     │     │
1 (d=3): ──────────0→1───0→2───1→0───1→2───2→0───2→1───M───
                   │     │     │     │     │     │
2 (d=3): ───(+2)───0→1───0→2───1→0───1→2───2→0───2→1───M───

In [30]:
cirq.Simulator().run(
    test_circuit_3,
    repetitions = 10
)

(0, 1)=0000000000
(0, 2)=1111111111
(1, 0)=1111111111
(1, 2)=0000000000
(2, 0)=0000000000
(2, 1)=0000000000
0 (d=3)=1111111111
1 (d=3)=0000000000
2 (d=3)=2222222222

In [31]:
def RzGates(cost_matrix, pow):
    for n1 in range(len(cost_matrix)):
        for n2 in range(len(cost_matrix)):
            if n1 != n2:
                yield cirq.ZPowGate(exponent = pow * cost_matrix[n1][n2])(
                    cirq.GridQubit(n1, n2)
                )

## Сама Цепь

In [32]:
class AncillaAndQuditsCircuit:

   def __init__(self, n_towns, depth, cost_matrix):

      self.n_towns = n_towns
      self.cost_matrix = cost_matrix
      self.cost_matrix_flattened = cost_matrix.ravel()[~np.isnan(cost_matrix.ravel())]
      # Flattened и без NaN (NaN стоят на диагональных элементах)
   
      self.circuit = cirq.Circuit()
      self.simulator = cirq.Simulator()

      self.flag_qubits = [cirq.GridQubit(n1, n2) for n1 in range(4) for n2 in range(4) if n1 != n2]

    # Инициализация единичной перестановкой
      for n in range(n_towns):
        self.circuit.append(ModuloPlus(n_towns, n)(cirq.LineQid(n, n_towns)))
            

    # Перестановка, близкая к полной
      self.circuit.append(iSwapAllQudits(n_towns, n_towns, 1/3)(
        *cirq.LineQid.range(n_towns, dimension = n_towns)
      ))

    # Параметрические гейты
      self._mixing_powers = [f'm{i}' for i in range(depth)]
      mixing_powers = sympy.symbols(self._mixing_powers)

      self._hamilt_powers = [f'h{i}' for i in range(depth)]
      hamilt_powers = sympy.symbols(self._hamilt_powers)

      self._angle_names = self._hamilt_powers + self._mixing_powers

      for p in range(depth):

        self.circuit.append(CNOTSequenceForAllPossibleEdges(n_towns))
        self.circuit.append(RzGates(cost_matrix, hamilt_powers[p]))
        self.circuit.append(reversed(list(
           CNOTSequenceForAllPossibleEdges(n_towns))
         ))

        self.circuit.append(iSwapAllQudits(n_towns, n_towns, mixing_powers[p])(
          *cirq.LineQid.range(n_towns, dimension = n_towns)
        ))

      # Измерение
      self.circuit.append(CNOTSequenceForAllPossibleEdges(n_towns))


   def measure_all(self, angles, repetitions = 1):

      params = cirq.ParamResolver({
         name: value for (name, value) in
         zip(self._angle_names, angles)
      })

      return self.simulator.run(
         cirq.Circuit(
            self.circuit,
            cirq.Moment(cirq.measure_each(
               *cirq.LineQid.range(self.n_towns, dimension = self.n_towns),
               *self.flag_qubits
            ))
         ), params, repetitions
      )


   def measure_expectations(self, angles):
      params = cirq.ParamResolver({
         name: value for (name, value) in
         zip(self._angle_names, angles)
      })

      return np.real(self.simulator.simulate_expectation_values(
         self.circuit,
         [(1 - cirq.Z(q)) / 2 for q in self.flag_qubits],
         # <0|cirq.Z|0> = +1
         params,
      ))


   def cost_from_expectation(self, angles):
      return np.dot(
         self.measure_expectations(angles),
         self.cost_matrix_flattened
      )


   def optimize_expectation(self, 
                           optimizer = differential_evolution,
                           **optimizer_kwargs):

      if optimizer == basinhopping or optimizer == minimize:
         if 'x0' not in optimizer_kwargs:
            optimizer_kwargs['x0'] = np.random.random(len(self._angle_names))
      else:
         if 'bounds' not in optimizer_kwargs:
            optimizer_kwargs['bounds'] = [(0, 1) for _ in self._angle_names]

      return optimizer(self.cost_from_expectation, **optimizer_kwargs)


   def __str__(self):
      return self.circuit.__str__()

   def _repr_pretty_(self, *args):
      "Text output in Jupyter"
      return self.circuit._repr_pretty_(*args) 

In [104]:
# 1.5 min
AncillaAndQuditsCircuit(4, 2, cost_matrix_4).optimize_expectation(shgo)

     fun: 1.2248616063631022
    funl: array([1.22486161])
 message: 'Optimization terminated successfully.'
    nfev: 115
     nit: 2
   nlfev: 98
   nlhev: 0
   nljev: 7
 success: True
       x: array([0.50152831, 0.50084527, 0.5       , 0.5       ])
      xl: array([[0.50152831, 0.50084527, 0.5       , 0.5       ]])

In [85]:
AncillaAndQuditsCircuit(4, 1, cost_matrix_4).circuit

(0, 1): ────────────────────────────────────X─────Z^(0.35271991*h0)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X──────────────X───────────────────────────────────────────────────────────────────────
                                            │                                                                                                                                                                                                                                                                                                                     │              │
(0, 2): ────────────────────────────────────┼─────X───────────────────Z^(0.96262685*h0)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────┼──────────────┼─────X─────────────────────────────────────────────────────────────────
                                            │     │                                                                                                                                                                                                                                                                                                         │     │              │     │
(0, 3): ────────────────────────────────────┼─────┼───────────────────X───────────────────Z^(0.11727604*h0)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────┼─────┼──────────────┼─────┼─────X───────────────────────────────────────────────────────────
                                            │     │                   │                                                                                                                                                                                                                                                                               │     │     │              │     │     │
(1, 0): ────────────────────────────────────┼─────┼───────────────────┼───────────────────X───────────────────Z^(0.13505078*h0)─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────┼─────┼─────┼──────────────┼─────┼─────┼─────X─────────────────────────────────────────────────────
                                            │     │                   │                   │                                                                                                                                                                                                                                                     │     │     │     │              │     │     │     │
(1, 2): ────────────────────────────────────┼─────┼───────────────────┼───────────────────┼───────────────────X───────────────────Z^(0.63915344*h0)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X─────┼─────┼─────┼─────┼──────────────┼─────┼─────┼─────┼─────X───────────────────────────────────────────────
                                            │     │                   │                   │                   │                                                                                                                                                               

In [41]:
cost_matrix_5 = \
np.array([[np.nan    , 0.33401903, 0.18976653, 0.57694655, 0.11001075],
          [0.68260228, np.nan    , 0.23440849, 0.63405642, 0.05852297],
          [0.79378125, 0.84210502, np.nan    , 0.17264401, 0.74223499],
          [0.5569718 , 0.55289136, 0.53119631, np.nan    , 0.08094091],
          [0.08584468, 0.61697247, 0.69688509, 0.29508331, np.nan    ]])

In [43]:
# Never ends
# AncillaAndQuditsCircuit(5, 2, cost_matrix_5).measure_expectations(np.random.random(4))